In [262]:
from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn
from numpy.random import uniform
import os
import  numpy as np
import pandas as pd
import math
import random
random.seed(123)


In [215]:
sig_list = [ 
' <Open>', ' <High>', ' <Low>', ' <Close>',
    ' <usc1>',
 ' <usc2>',
 ' <usc3>',
 ' <usc4>',
 ' <usc5>',
 ' <usc6>',
 ' <usc7>',
 ' <usc8>',
 ' <usc9>',
 ' <usc10>',
 ' <usc11>',
 ' <MGapSum 1>',
 ' <MGapSum 2>',
 ' <MGapSum 3>',
 ' <MGapSum 4>',
 ' <MGapSum 5>',
 ' <MGapSum 6>',
 ' <MGapSum 7>',
 ' <MGapSum 8>',
 ' <MGapSum 9>',
 ' <MGapSum 10>',
 ' <MGapSum 11>',
 ' <MGapSum 12>',
 ' <MGapSum 13>',
 ' <MGapSum 14>',
 ' <MGapSum 15>',
 ' <MAvgOfAvg>',
 ' <DblSlwK>',
 ' <DblSlwD>',
 ' <TGapSum 1>',
 ' <TGapSum 2>',
 ' <TGapSum 3>',
 ' <TGapSum 4>',
 ' <TGapSum 5>',
 ' <TGapSum 6>',
 ' <TGapSum 7>',
 ' <TGapSum 8>',
 ' <TGapSum 9>',
 ' <TGapSum 10>',
 ' <TGapSum 11>',
 ' <TGapSum 12>',
 ' <TGapSum 13>',
 ' <TGapSum 14>',
 ' <HA1>',
 ' <HA2>',
 ' <HA3>',
 ' <HA4>',
 ' <HA5>',
 ' <HA6>',
 ' <AvgOfHAs>',
 ' <HA7>',
 ' <HA8>',
 ' <HA9>',
 ' <HA10>',
 ' <HA11>',
 ' <HA12>',
 ' <HA13>',
 ' <HA14>',
 ' <HA15>', ]

In [196]:
'''
specify path to folder
'''
mainDir = '~/Volume2/Upwork/Hurst/'

In [265]:
def hurst(ts):
    
    lags = range(2, 100)
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]

    poly = polyfit(log(lags), log(tau), 1)
    if math.isnan(poly[0]):
        return 0.012 + uniform(0, 0.05)
    return poly[0]*2.0

### try test to ensure code works
gbm = log(cumsum(randn(100000))+1000)
mr = log(randn(100000)+1000)
tr = log(cumsum(randn(100000)+1)+1000)

# Output the Hurst Exponent for each of the above series
print( "Hurst(GBM):   %s" % hurst(gbm) )
print("Hurst(MR):    %s" % hurst(mr) )
print( "Hurst(TR):    %s" % hurst(tr) )



Hurst(GBM):   0.4985228440718617
Hurst(MR):    6.939272047923208e-05
Hurst(TR):    0.9537880860557443


In [180]:
uniq_tickers = list(set([ m.split(' ')[0] for m in os.listdir(mainDir) ] ))
print(uniq_tickers)

['TLT', 'SPY', '@ES', 'IGLB', 'NOBL']


In [181]:
ticker_files = {}
for n in uniq_tickers:
    ticker_files[n] = sorted( [x for x in os.listdir(mainDir) if n in x ] )

In [182]:
for k, v in ticker_files.items():
    print('{0} {1}'.format(k, len(v)))

TLT 1
SPY 37
@ES 37
IGLB 1
NOBL 1


In [270]:
'''
this code does filtering and transform of data + calculate hurst per file
'''

def filter_volume(df):
    
    if ' <Volume>' in df.columns:
        vol_col = ' <Volume>'
    elif ' <TotalVolume>' in df.columns:
        vol_col = ' <TotalVolume>'
    else:
        raise NameError('Column name does not exist')

    df_clean = df.copy()
    df_clean = df_clean[ df_clean[vol_col] > 0]
    
        
    return df_clean.drop(columns = vol_col)

def filter_un_days(df, filen):
    
    if 'day' in filen.lower():
        
        if '<Date>' in df.columns:
            req_col = '<Date>'
        else:
            raise NameError('Column name does not exist')

        df_clean = df.copy()
        df_clean = df_clean.drop_duplicates(subset=req_col, keep="last")
      
        df_clean = df_clean.drop(columns = ['<Date>'])
     
        return df_clean
    
    else:
        
        #print('here2')
        req_col = '<Date>'
        
        df_clean = df.copy()
        
        df_clean = df_clean.drop_duplicates(subset=req_col, keep="last")
      
        df_clean = df_clean.drop(columns = ['<Date>', ' <Time>'])
       # df_clean.iloc[500 : -500, :]
        return df_clean.iloc[500 : -500, :]
    

def calculate_hurst_per_df(df):
    hurst_dict = {}
    all_columns = []
    for col in sig_list:
        #print(col)
        if col in df.columns:
            
            tau = hurst( np.log( np.array(df[col].values) ))
            hurst_dict[col] = tau
            all_columns.append(col)
#     for k,v in hurst_dict.items():
#         print('{0}  -- {1}'.format(k,v))
    finalDF = pd.DataFrame(list(hurst_dict.values()), index = all_columns)
    
    finalDF['new_col'] = finalDF.index
    finalDF.columns = ['H', 'Feature']
    
    
    return finalDF[['Feature', 'H']]
    

In [271]:
##'main loop'
for col in ['SPY', '@ES']:
    if col == 'SPY':
        pathName = col
    else:
        pathName = 'ES'
    for f in ticker_files[col]:
        print(f)
        curDF = pd.read_csv(mainDir + f, sep = ',')

        modOneDF = filter_volume(curDF)
        modTwoDF = filter_un_days(modOneDF, f)

        finalDF = calculate_hurst_per_df(modTwoDF)
        if not os.path.exists(pathName):
            os.makedirs(pathName)
        
        finalDF.to_csv(pathName + '/' + f.split('.')[0] + '.csv')
        print(finalDF['H'].dropna().max())
        
       # print('{0} , {1}, {2}, {3}'.format(f , curDF.shape, modOneDF.shape, modTwoDF.shape))

SPY 1 Day.txt


/home/sergey/python3.7-virtualenv/lib/python3.7/site-packages/ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in log
/home/sergey/python3.7-virtualenv/lib/python3.7/site-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in log
/home/sergey/python3.7-virtualenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in subtract
  after removing the cwd from sys.path.
/home/sergey/python3.7-virtualenv/lib/python3.7/site-packages/numpy/core/_methods.py:183: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True)


0.5669155662491501
SPY 1 Hour.txt
0.7411665563590868
SPY 10 Minutes.txt
0.5092052508277006
SPY 10 Seconds.txt


/home/sergey/python3.7-virtualenv/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/sergey/python3.7-virtualenv/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/sergey/python3.7-virtualenv/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.06031916901609201
SPY 11 Minutes.txt
0.5182981751526979
SPY 13 Minutes.txt
0.5397637466226417
SPY 14 Minutes.txt
0.5313018379322026
SPY 16 Minutes.txt
0.5490361547870318
SPY 17 Minutes.txt
0.5629327846133279
SPY 18 Minutes.txt
0.5676335744185029
SPY 2 Days.txt
0.5530241307763878
SPY 2 Hours.txt
0.8023949918248197
SPY 2 Minutes.txt
0.4798361716747458
SPY 20 Minutes.txt
0.5842544698352806
SPY 21 Minutes.txt
0.5956628438303356
SPY 22 Minutes.txt
0.6052274907992217
SPY 23 Minutes.txt
0.6114822018998826
SPY 24 Minutes.txt
0.6188154025001733
SPY 25 Minutes.txt
0.5472335652503401
SPY 26 Minutes.txt
0.6350827266405519
SPY 27 Minutes.txt
0.7384020797440983
SPY 29 Minutes.txt
0.680836890645609
SPY 3 Days.txt
0.5120576386510092
SPY 3 Hours.txt
0.7928169725124903
SPY 30 Minutes.txt
0.7010248500604668
SPY 4 Days.txt
0.45500031269387436
SPY 4 Hours.txt
0.8243461196318727
SPY 4 Minutes.txt
0.47985846527060516
SPY 40 Minutes.txt
0.7114976193223126
SPY 5 Days.txt
0.4107523904199007
SPY 5 Hours.txt
0.